# iPython Nutrition Notebook

This notebook was created to make it easier to understand how several behavioral variables affect your body's composition. Please add or remove from it what you like, as it was only ever intended to aid the otherwise obnoxiously tedious activity that is macro tracking.

## Initial setup

In [ ]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt

#Start MongoDB Client
client = MongoClient()
db = client.primer
coll = db.dataset

#FOODS

#Simple Foods
empty = np.array([0.,0.,0.,0.])
Egg = np.array([78., 5., 0.6, 6.])
Avocado_whole = np.array([234., 21., 12., 2.9])
Sour_Cream_1tbsp = np.array([23., 2.4, 0.3, 0.2])
Spinach = np.array([7., 0.1, 1.1, 0.9])
Chicken_1oz = np.array([32.5, 0.75, 0, 6.5])
Peanut_Butter_2tbsp = np.array([200., 16., 6., 8.])
Refried_Beans_1serving = np.array([195, 0.89, 33.7, 10.64])
Trail_Mix_hfcup = np.array([360, 24., 31.2, 12.])
Protein_Powder_1scoop = np.array([140., 2., 6., 24.])
Oatmeal_hfcup = np.array([150.0, 3.0, 27.0, 5.0])
Sweet_Potato_1oz = np.array([86./3.5274, 0.1/3.5274, 20./3.5274, 1.6/3.5274])
White_Rice_1cp = np.array([206, 0.4, 45.0, 4.3])
One_Percent_Milk_1cup = np.array([130, 2.5, 16., 11.])
Cottage_Cheese_hfcup = np.array([80.0, 0.0, 6.0, 13.0])

#Protein
Amplified_Mass_1scoop = np.array([187.5, 1.5, 31, 12.5])
Protein_Powder_MochaCappuccino = np.array([120, 1.5, 3., 24.])
Protein_Powder_DoubleChocolate = np.array([120, 1., 3., 24.])

#Mealy Foods
chicken_serving_small = 8*Chicken_1oz
chicken_serving_large = 10*Chicken_1oz
halfAnAvocado = np.multiply(0.5, Avocado_whole)
quarterAnAvocado = np.multiply(0.25, Avocado_whole)
omelette = 6*Egg + halfAnAvocado + 6*Sour_Cream_1tbsp + Spinach
omeletteLessFat = 6*Egg + quarterAnAvocado + Spinach
oatmeal = 2*Oatmeal_hfcup
halfMass = 2*Amplified_Mass_1scoop
halfMassWithMilk = halfMass + One_Percent_Milk_1cup

## Tracking

In [ ]:
#Beginning of day tracking
weight = 0
bodyfatpercentage = 0
sleep = {"hours": 0, "minutes": }

#Meal tracking
firstMeal = empty
print("First Meal Calories: " + `firstMeal[0]`)
print("First Meal Fat: " + `firstMeal[1]`)
print("First Meal Carbs: " + `firstMeal[2]`)
print("First Meal Protein: " + `firstMeal[3]` + "\n")

secondMeal = empty
print("Second Meal Calories: " + `secondMeal[0]`)
print("Second Meal Fat: " + `secondMeal[1]`)
print("Second Meal Carbs: " + `secondMeal[2]`)
print("Second Meal Protein: " + `secondMeal[3]` + "\n")

thirdMeal = empty
print("Third Meal Calories: " + `thirdMeal[0]`)
print("Third Meal Fat: " + `thirdMeal[1]`)
print("Third Meal Carbs: " + `thirdMeal[2]`)
print("Third Meal Protein: " + `thirdMeal[3]` + "\n")

fourthMeal = empty
print("Fourth Meal Calories: " + `fourthMeal[0]`)
print("Fourth Meal Fat: " + `fourthMeal[1]`)
print("Fourth Meal Carbs: " + `fourthMeal[2]`)
print("Fourth Meal Protein: " + `fourthMeal[3]` + "\n")

total = firstMeal + secondMeal + thirdMeal + fourthMeal
print("Total Calories: " + `total[0]`)
print("Total Fat: " + `total[1]`)
print("Total Carbs: " + `total[2]`)
print("Total Protein: " + `total[3]`)


## Data retrieval

In [ ]:
#lists to store tracked variables
days = []
calories = []
fat = []
carbs = []
protein = []
sleep = []
weight = []

#setup for mongo search
daysShown = 7
earliestDate = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - daysShown
cursor = db.macros.find( { 'date': { '$gt': earliestDate } } )

#add each entry's variables to the respective lists 
for document in cursor:
    days.append(document['date'])
    calories.append(document['macros']['calories'])
    fat.append(document['macros']['fat'])
    carbs.append(document['macros']['carbohydrates'])
    protein.append(document['macros']['protein'])
    weight.append(document['weight'])
    sleep.append((document['sleep']['hours'],document['sleep']['minutes']))
    print(document)


## Graphing.

In [ ]:
today = dt.date.today()


f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.plot(days, calories, color='y')
ax2.plot(days, weight, color='b')
ax3.plot(days, carbs, color='r')
ax4.plot(days, protein, color='g')

ax1.set_ylim([min(calories), max(calories)*1.5])
ax2.set_ylim([min(weight), max(weight)*1.1])
ax3.set_ylim([min(carbs), max(carbs)*1.5])
ax4.set_ylim([min(protein), max(protein)*1.3])

ax1.set_title('Calories')
ax2.set_title('Weight')
ax3.set_title('Carbohydrates')
ax4.set_title('Protein')


plt.savefig(`today.month` + "-" + `today.day` + "-" + `today.year` + ".png")


## Database Insertion

In [ ]:


daysSinceEpoch = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - 1

result = db.macros.insert_one(
    {
        "date": daysSinceEpoch,
        "macros": {
            "calories": total[0],
            "fat": total[1],
            "carbohydrates": total[2],
            "protein": total[3]
        },
        "weight": weight,
        "bodyfat%": bodyfatpercentage,
        "sleep": sleep
    }
)